## RNAseq notebook 3.2: SAM files and read counting
This notebook features examples on how to work with the sequence alignment map file, and how to derive gene counts from mapped reads.  
**Notes**
- full SAM files tend to be large so bash manipulation can take some time (typically minutes)
- not all SAM attributes will be found in all SAM files

**Find where a specific read aligned**  
Example: 1000th read

```bash
%%bash  
cd ../Downloads/hisat_out  
grep SRR5454079.1000 ./SRR5454079.sam | head -n 1
```

**Print the first few aligned reads**
```bash
%%bash
cd ../Downloads/hisat_out
awk '/^SRR/' SRR5454079.sam | head
```

**Print the first few reads that aligned to the mitochondrial genome**  
```bash
%%bash  
cd ../Downloads/hisat_out  
awk '{if ($3 == "MT") {print}}' SRR5454079.sam | head
```

**Exercises**   
1) How many reads mapped to chromosome 20?  
2) Find the 76th read in the SAM file. Where did it map in the human genome? Now use blastn to map the read. Do the results agree with each other?  
3) Inspect the reference genome details in the SAM header. Beyond chromosomes, what else is included in the reference?

In [12]:
%%bash
cd ../../RNAseq/hisat_out/
awk '{if ($3 == "20") {print}}' SRR5454079_1.sam | wc -l

464329


In [34]:
%%bash
cd ../../RNAseq/hisat_out/
grep SRR5454079.76 ./SRR5454079_1.sam | head -n 1

SRR5454079.76	16	20	327964	60	50M	*	0	0	ATACAGCGGGAAAAACTGAGGCACTTTGGTGCTAGGGGTTTGGGACTGAN	JJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJFFAA#	AS:i:-2	ZS:i:-2	XN:i:0	XM:i:1	XO:i:0	XG:i:0	NM:i:1	MD:Z:49G0	YT:Z:UU	NH:i:1


In [13]:
%%bash
cd ../../RNAseq/hisat_out/
head -n 100 ./SRR5454079_1.sam

@HD	VN:1.0	SO:unsorted
@SQ	SN:1	LN:248956422
@SQ	SN:10	LN:133797422
@SQ	SN:11	LN:135086622
@SQ	SN:12	LN:133275309
@SQ	SN:13	LN:114364328
@SQ	SN:14	LN:107043718
@SQ	SN:15	LN:101991189
@SQ	SN:16	LN:90338345
@SQ	SN:17	LN:83257441
@SQ	SN:18	LN:80373285
@SQ	SN:19	LN:58617616
@SQ	SN:2	LN:242193529
@SQ	SN:20	LN:64444167
@SQ	SN:21	LN:46709983
@SQ	SN:22	LN:50818468
@SQ	SN:3	LN:198295559
@SQ	SN:4	LN:190214555
@SQ	SN:5	LN:181538259
@SQ	SN:6	LN:170805979
@SQ	SN:7	LN:159345973
@SQ	SN:8	LN:145138636
@SQ	SN:9	LN:138394717
@SQ	SN:MT	LN:16569
@SQ	SN:X	LN:156040895
@SQ	SN:Y	LN:57227415
@SQ	SN:KI270728.1	LN:1872759
@SQ	SN:KI270727.1	LN:448248
@SQ	SN:KI270442.1	LN:392061
@SQ	SN:KI270729.1	LN:280839
@SQ	SN:GL000225.1	LN:211173
@SQ	SN:KI270743.1	LN:210658
@SQ	SN:GL000008.2	LN:209709
@SQ	SN:GL000009.2	LN:201709
@SQ	SN:KI270747.1	LN:198735
@SQ	SN:KI270722.1	LN:194050
@SQ	SN:GL000194.1	LN:191469
@SQ	SN:KI270742.1	LN:186739
@SQ	SN:GL000205.2	LN:185591
@SQ	SN:GL000195.1	LN:182896
@SQ	SN:KI270736.1	LN:181920
@SQ	

**Check how many reads were uncounted due to multimapping (alignment not unique)**
```bash
%%bash
cd Unit2-RNAseq/data
tail SRR5454102_genecounts.txt
```

In [2]:
%%bash
cd data
tail SRR5454102_genecounts.txt

ENSG00000285990	0
ENSG00000285991	1
ENSG00000285992	0
ENSG00000285993	0
ENSG00000285994	2
__no_feature	3712490
__ambiguous	589511
__too_low_aQual	0
__not_aligned	1202625
__alignment_not_unique	7178464


**Check how many Ensembl genes have zero expression**  
Spot and correct the mistake
```bash
%%bash
cd Unit2-RNAseq/data
awk '{if ($2 == 0) print }' SRR5454102_genecounts.txt  | wc -l
```

In [7]:
%%bash
cd data
awk '{if ($2 == 0 && $1 ~ /^ENS/) print }' SRR5454102_genecounts.txt | wc -l
awk '{if ($2 == 0) print }' SRR5454102_genecounts.txt | wc -l

31131
31132


**HOMEWORK**  
1) Use awk to check the number of columns in the SAM file for all rows and print only the unique column counts. *HINT*: revisit Unit 1   
2) Count how many reads from SRR5454079 mapped to chromosome 20 with 2 soft-clipped bases at the start of the read. *HINT*: Consult the SAM documentation on CIGAR strings.  
3) Using the human transcriptome annotations by Ensembl, calculate counts per gene in the bam file for SRR5454079 and print the first 10 lines (use -s reverse)

In [31]:
%%bash
cd ../../RNAseq/hisat_out/
awk '{print NF}' SRR5454079_1.sam | sort -n | uniq

3
12
13
20
21
22


In [45]:
%%bash
cd ../../RNAseq/hisat_out/
awk '{if ($3==20 && $6 ~ /^2S/) print}' SRR5454079_1.sam | wc -l

5725


In [49]:
%%bash
cd ../../RNAseq/hisat_out/
htseq-count -f bam -s reverse SRR5454079_1.sort.bam ../Ensembl/Homo_sapiens.GRCh38.94.gtf.gz > SRR5454079_1_counts.txt
head SRR5454079_1_counts.txt

ENSG00000000003	433
ENSG00000000005	1
ENSG00000000419	247
ENSG00000000457	194
ENSG00000000460	168
ENSG00000000938	5
ENSG00000000971	1
ENSG00000001036	224
ENSG00000001084	427
ENSG00000001167	1040
